In [1]:
import torch
import torch.utils.data as data
import pandas as pd
from src.error_measures import get_accuracy, get_error_measures
from src.models import TwoLayerPerceptron, SynapticSNN, LeakySNN, DoubleLeakySNN
from src.train_model import training_loop

In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

## Data

In [3]:
data06 = pd.read_csv("../data/processed/target06.csv")
data06.drop(columns=["weekday"], inplace=True)

In [4]:
all_data = data.TensorDataset(torch.from_numpy((data06.values[:,:-1] - data06.values[:,:-1].min(0)) / data06.values[:,:-1].ptp(0)).float(),
                              torch.from_numpy(data06.values[:,-1]).float())  # with normalization
train_dataset, test_dataset, valid_dataset = torch.utils.data.random_split(all_data, (round(0.75 * len(all_data)), round(0.15 * len(all_data)), round(0.1 * len(all_data))))

In [5]:
train_loader = data.DataLoader(train_dataset, batch_size=32, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=256, shuffle=False, drop_last=False)
valid_loader = data.DataLoader(valid_dataset, batch_size=256, shuffle=False, drop_last=False)

### Simple regressor

In [6]:
model1 = TwoLayerPerceptron(num_inputs=8, num_hidden=2500, num_outputs=1)
print(model1)

TwoLayerPerceptron(
  (lin1): Linear(in_features=8, out_features=2500, bias=True)
  (relu): LeakyReLU(negative_slope=0.01)
  (lin2): Linear(in_features=2500, out_features=1, bias=True)
)


In [7]:
training_loop(model1, train_loader, valid_loader, device, num_epochs=25, validation=True)

  4%|▍         | 1/25 [00:02<01:04,  2.70s/it]

MAE: 8.535572223994084, MSE:148.26054984749038,
RMSE: 12.176228884490074, IA: 0.6112025767774917


 16%|█▌        | 4/25 [00:03<00:14,  1.41it/s]

MAE: 6.856809207056299, MSE:97.94179492301349,
RMSE: 9.89655469964237, IA: 0.8436394488252793


 28%|██▊       | 7/25 [00:05<00:08,  2.07it/s]

...

MAE: 6.762346799111779, MSE:93.23395520013678,
RMSE: 9.65577315392904, IA: 0.8571457368571315


 88%|████████▊ | 22/25 [00:11<00:01,  2.28it/s]

MAE: 6.612820816040039, MSE:93.42147497714357,
RMSE: 9.66547851775294, IA: 0.8607854606319884


100%|██████████| 25/25 [00:13<00:00,  1.91it/s]

MAE: 6.553190719736794, MSE:91.64476611795662,
RMSE: 9.573127290387223, IA: 0.8607275388422182
Accuracy on validation dataset: 47.75%


In [8]:
print(f'Accuracy on test set: {get_accuracy(model1, test_loader, device=device, pct_close=0.25)*100}%')
get_error_measures(model1, test_loader, device=device, print_e=True)

Accuracy on test set: 50.31%
MAE: 6.323844388379889, MSE:85.51697446651542,
RMSE: 9.24753883292822, IA: 0.885496378614432


(6.323844388379889, 85.51697446651542, 9.24753883292822, 0.885496378614432)

### SNN with Leaky layer

In [6]:
model2 = LeakySNN(num_inputs=8, num_hidden=2500, num_outputs=1)
print(model2)

TwoLayerLeaky(
  (fc1): Linear(in_features=8, out_features=2500, bias=True)
  (lif): Leaky()
  (fc2): Linear(in_features=2500, out_features=1, bias=False)
)


In [7]:
training_loop(model2, train_loader, valid_loader, device, num_epochs=25, validation=True)

  4%|▍         | 1/25 [00:06<02:39,  6.65s/it]

MAE: 9.601514458794126, MSE:198.0351522101949,
RMSE: 14.072496303435113, IA: 0.43912075107015947


 16%|█▌        | 4/25 [00:20<01:42,  4.86s/it]

MAE: 7.302647416440049, MSE:116.41956910854,
RMSE: 10.789790040058241, IA: 0.82116843201532


 28%|██▊       | 7/25 [00:34<01:24,  4.71s/it]

...

MAE: 6.924184981384719, MSE:101.33744765718679,
RMSE: 10.066650269935218, IA: 0.8577013479348683


 88%|████████▊ | 22/25 [01:43<00:14,  4.70s/it]

MAE: 7.053109839610282, MSE:103.10731936931087,
RMSE: 10.15417743440161, IA: 0.8515389280279116


100%|██████████| 25/25 [01:57<00:00,  4.70s/it]

MAE: 7.163364157649134, MSE:99.50560779319964,
RMSE: 9.975249760943314, IA: 0.8627212049638938
Accuracy on validation dataset: 45.32%


In [8]:
print(f'Accuracy on test set: {get_accuracy(model2, test_loader, device=device, pct_close=0.2)*100}%')
get_error_measures(model2, test_loader, device=device, print_e=True)

Accuracy on test set: 37.669999999999995%
MAE: 6.69671032939376, MSE:86.44064148634061,
RMSE: 9.297345937757754, IA: 0.853880390569408


(6.69671032939376, 86.44064148634061, 9.297345937757754, 0.853880390569408)

### SNN with Synaptic layer

In [12]:
model3 = SynapticSNN(num_inputs=8, num_hidden=2500, num_outputs=1)
print(model3)

TwoLayerSynaptic(
  (fc1): Linear(in_features=8, out_features=2500, bias=True)
  (lif): Synaptic()
  (fc2): Linear(in_features=2500, out_features=1, bias=True)
)


In [13]:
training_loop(model3, train_loader, valid_loader, device, num_epochs=20, validation=True)

  5%|▌         | 1/20 [00:19<06:14, 19.72s/it]

MAE: 7.818231159276356, MSE:120.42788208949321,
RMSE: 10.97396382760091, IA: 0.7456003500414856


 20%|██        | 4/20 [01:17<05:11, 19.46s/it]

MAE: 6.625340978258607, MSE:92.43018684795527,
RMSE: 9.614061932812543, IA: 0.8547333987767537


 35%|███▌      | 7/20 [02:17<04:14, 19.60s/it]

...

MAE: 6.703999995909674, MSE:94.98943056162022,
RMSE: 9.746252128978616, IA: 0.862615379782494


 95%|█████████▌| 19/20 [06:03<00:19, 19.11s/it]

MAE: 6.793549091140659, MSE:92.17983708145347,
RMSE: 9.601033125734618, IA: 0.8612399712229653


100%|██████████| 20/20 [06:21<00:00, 19.09s/it]


Accuracy on validation dataset: 48.67%


In [14]:
print(f'Accuracy on test set: {get_accuracy(model3, test_loader, device=device, pct_close=0.25)*100}%')
get_error_measures(model3, test_loader, device=device, print_e=True)

Accuracy on test set: 50.62%
MAE: 6.301764238780746, MSE:82.199676836497,
RMSE: 9.066403743298498, IA: 0.8888638637486594


(6.301764238780746, 82.199676836497, 9.066403743298498, 0.8888638637486594)

### SNN with two Leaky layers

In [15]:
model4 = DoubleLeakySNN(num_inputs=8, num_hidden=2500, num_outputs=1)
print(model4)

ThreeLayerLeaky(
  (lin1): Linear(in_features=8, out_features=2500, bias=True)
  (lif1): Leaky()
  (lif2): Leaky()
  (lin2): Linear(in_features=2500, out_features=1, bias=True)
)


In [16]:
training_loop(model4, train_loader, valid_loader, device, num_epochs=20, validation=True)

  5%|▌         | 1/20 [00:09<02:53,  9.15s/it]

MAE: 9.45273986882557, MSE:180.17615649733347,
RMSE: 13.422971224633296, IA: 0.396622310510242


 20%|██        | 4/20 [00:36<02:22,  8.89s/it]

MAE: 7.473445032924586, MSE:106.21734866091151,
RMSE: 10.3061801197588, IA: 0.8342725805137609


 35%|███▌      | 7/20 [01:02<01:56,  8.96s/it]

...

MAE: 7.0720461867448225, MSE:100.54827395166143,
RMSE: 10.027376224699132, IA: 0.8446595485319661


 95%|█████████▌| 19/20 [02:48<00:08,  8.96s/it]

MAE: 6.790115591280722, MSE:97.14783198677837,
RMSE: 9.856359976521675, IA: 0.85194177791481


100%|██████████| 20/20 [02:56<00:00,  8.83s/it]


Accuracy on validation dataset: 44.97%


In [17]:
print(f'Accuracy on test set: {get_accuracy(model4, test_loader, device=device, pct_close=0.25)*100}%')
get_error_measures(model4, test_loader, device=device, print_e=True)

Accuracy on test set: 49.61%
MAE: 6.509637902073206, MSE:90.49805812315023,
RMSE: 9.51304673189143, IA: 0.8740836822504967


(6.509637902073206, 90.49805812315023, 9.51304673189143, 0.8740836822504967)